![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [79]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [80]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [81]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [82]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
df

,name
0,airports
1,airlines
2,routes


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [83]:
cursor.execute("SELECT * FROM airports")

#### Use description

In [84]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [85]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [86]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [87]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


In [88]:
cursor.execute(select* from routes)

SyntaxError: invalid syntax (<ipython-input-88-aaa642a0900e>, line 1)

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

In [89]:
cursor.execute("""SELECT * FROM airlines;""").fetchall()

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N'),
 (5,
  '6',
  '223 Flight Unit State Airline',
  '\\N',
  None,
  'CHD',
  'CHKALOVSK-AVIA',
  'Russia',
  'N'),
 (6,
  '7',
  '224th Flight Unit',
  '\\N',
  None,
  'TTF',
  'CARGO UNIT',
  'Russia',
  'N'),
 (7,
  '8',
  '247 Jet Ltd',
  '\\N',
  None,
  'TWF',
  'CLOUD RUNNER',
  'United Kingdom',
  'N'),
 (8, '9', '3D Aviation', '\\N', None, 'SEC', 'SECUREX', 'United States', 'N'),
 (9,
  '10',
  '40-Mile Air',
  '\\N',
  'Q5',
  'MLA',
  'MILE-AIR',
  'United States',
  'Y'),
 (10, '11', '4D Air', '\\N', None, 'QRT', 'QUARTET', 'Thailand', 'N'),
 (11,
  '12',
  '611897 

In [90]:
import pandas as pd

In [91]:
df=cursor.execute("""SELECT * FROM airlines WHERE (country="United Kingdom" and active="Y") ORDER BY country DESC;""").fetchall()

In [92]:
df_2=pd.DataFrame(df)

In [93]:
df_2.columns=[x[0] for x in cursor.description]

In [94]:
df_2.head()

,index,id,name,alias,iata,icao,callsign,country,active
0,111,112,Astraeus,\N,5W,AEU,FLYSTAR,United Kingdom,Y
1,491,492,Air Southwest,\N,None,WOW,SWALLOW,United Kingdom,Y
2,507,508,Aurigny Air Services,\N,GR,AUR,AYLINE,United Kingdom,Y
3,564,565,Air Wales,\N,6G,AWW,RED DRAGON,United Kingdom,Y
4,664,665,AD Aviation,\N,None,VUE,FLIGHTVUE,United Kingdom,Y


In [95]:
cursor.execute("Select distinct country from airports;").fetchall()

[('Papua New Guinea',),
 ('Greenland',),
 ('Iceland',),
 ('Canada',),
 ('Algeria',),
 ('Benin',),
 ('Burkina Faso',),
 ('Ghana',),
 ("Cote d'Ivoire",),
 ('Nigeria',),
 ('Niger',),
 ('Tunisia',),
 ('Togo',),
 ('Belgium',),
 ('Germany',),
 ('United States',),
 ('Estonia',),
 ('Finland',),
 ('United Kingdom',),
 ('Guernsey',),
 ('Jersey',),
 ('Isle of Man',),
 ('Falkland Islands',),
 ('Netherlands',),
 ('Ireland',),
 ('Denmark',),
 ('Faroe Islands',),
 ('Luxembourg',),
 ('Norway',),
 ('Poland',),
 ('Sweden',),
 ('South Africa',),
 ('Botswana',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Swaziland',),
 ('Central African Republic',),
 ('Equatorial Guinea',),
 ('Saint Helena',),
 ('Mauritius',),
 ('British Indian Ocean Territory',),
 ('Cameroon',),
 ('Zambia',),
 ('Comoros',),
 ('Mayotte',),
 ('Reunion',),
 ('Madagascar',),
 ('Angola',),
 ('Puerto Rico',),
 ('Gabon',),
 ('Sao Tome and Principe',),
 ('Mozambique',),
 ('Seychelles',),
 ('Chad',),
 ('Zimbabwe',),
 ('Malawi',),
 ('Le

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [103]:
# cursor.execute("Select r.*, latitude, longitude from airports a join routes r on  a.id =r.source_id ;").fetchall()
df = pd.read_sql_query("Select r.*, latitude, longitude from airports a join routes r on  a.id =r.source_id ;", conn)


DatabaseError: Execution failed on sql 'Select r.*, latitude, longitude from airports a join routes r on  a.id =r.source_id ;': no such table: airports

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [97]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [98]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
349,A443876,Max,05/06/2019 04:56:00 PM,05/06/2019 04:56:00 PM,03/21/2006,Adoption,NaN,Cat,Neutered Male,13 years,Domestic Shorthair Mix,Blue Tabby/White
673,A793215,Max,04/30/2019 11:49:00 AM,04/30/2019 11:49:00 AM,04/21/2017,Return to Owner,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Black/White
823,A731802,Max,04/26/2019 06:17:00 PM,04/26/2019 06:17:00 PM,07/27/2015,Adoption,NaN,Dog,Neutered Male,3 years,Maltese Mix,White
824,A789320,Max,04/26/2019 06:01:00 PM,04/26/2019 06:01:00 PM,08/19/2017,Adoption,NaN,Dog,Neutered Male,1 year,Pit Bull Mix,White/Black
1162,A793073,Max,04/19/2019 01:55:00 PM,04/19/2019 01:55:00 PM,04/19/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Boxer Mix,Brown/Tricolor


In [99]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794145,NaN,05/13/2019 04:36:00 PM,05/13/2019 04:36:00 PM,03/04/2019,Transfer,Partner,Dog,Intact Male,2 months,Shih Tzu Mix,White/Brown
2,A789190,*Dublin,05/13/2019 12:19:00 PM,05/13/2019 12:19:00 PM,02/17/2016,Adoption,NaN,Dog,Neutered Male,3 years,Australian Cattle Dog/Labrador Retriever,Red Tick/White
3,A794772,Rocky,05/13/2019 12:19:00 PM,05/13/2019 12:19:00 PM,05/12/2018,Return to Owner,NaN,Dog,Intact Male,1 year,Labrador Retriever,Brown/White
5,A794788,Rocco,05/13/2019 11:59:00 AM,05/13/2019 11:59:00 AM,08/13/2018,Return to Owner,NaN,Dog,Neutered Male,8 months,Miniature Pinscher,Black/Tan
6,A794472,NaN,05/13/2019 11:45:00 AM,05/13/2019 11:45:00 AM,05/08/2017,Transfer,Partner,Dog,Spayed Female,2 years,Bichon Frise Mix,White


## Goal 6: Transfering from sqlite to pandas

In [122]:
conn = sqlite3.connect("Chinook_Sqlite.sqlite")
c=conn.cursor()
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
df

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [132]:
# conn.execute("SELECT primary_key FROM Chinook_Sqlite.sqlite;")


OperationalError: no such column: type

In [138]:
c.execute("PRAGMA table_info(Album)")
info = c.fetchall()
print(*info, sep = "\n") 

(0, 'AlbumId', 'INTEGER', 1, None, 1)
(1, 'Title', 'NVARCHAR(160)', 1, None, 0)
(2, 'ArtistId', 'INTEGER', 1, None, 0)


In [139]:
c.execute("PRAGMA foreign_key_list (Track)")
info = c.fetchall()
print(*info, sep = "\n") 

(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE')
(2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')


In [141]:
c.execute("PRAGMA table_info(Artist)")
info = c.fetchall()
print(*info, sep = "\n") 

(0, 'ArtistId', 'INTEGER', 1, None, 1)
(1, 'Name', 'NVARCHAR(120)', 0, None, 0)


In [152]:
c.execute("PRAGMA table_info(Invoice)")
info = c.fetchall()
print(*info, sep = "\n") 

(0, 'InvoiceId', 'INTEGER', 1, None, 1)
(1, 'CustomerId', 'INTEGER', 1, None, 0)
(2, 'InvoiceDate', 'DATETIME', 1, None, 0)
(3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0)
(4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0)
(5, 'BillingState', 'NVARCHAR(40)', 0, None, 0)
(6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0)
(7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0)
(8, 'Total', 'NUMERIC(10,2)', 1, None, 0)


In [157]:
c.execute("""SELECT max(InvoiceDate), min(InvoiceDate) from Invoice;""").fetchall()


[('2013-12-22 00:00:00', '2009-01-01 00:00:00')]

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
Chinook_Sqlite.sqlite

## Reflection